In [1]:
import numpy as np
import pandas as pd
import cv2
import pytesseract
from glob import glob
import spacy
import re
import string

In [2]:
def cleanText(txt):
    whitespace = string.whitespace
    punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
    tableWhitespace = str.maketrans('','',whitespace)
    tablePunctuation = str.maketrans('','',punctuation)
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

In [116]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
### Load NER model
model_ner = spacy.load('./output/model-best/')

In [9]:
# Load Image
image = cv2.imread('./data/6.jpg')

# cv2.imshow('businesscard',image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# extract data using Pytesseract 
tessData = pytesseract.image_to_data(image)
# convert into dataframe
tessList = list(map(lambda x:x.split('\t'), tessData.split('\n')))
df = pd.DataFrame(tessList[1:],columns=tessList[0])
df.dropna(inplace=True) # drop missing values
df['text'] = df['text'].apply(cleanText)

# convet data into content
df_clean = df.query('text != "" ')
content = " ".join([w for w in df_clean['text']])
print(content)
# get prediction from NER model
doc = model_ner(content)

cell 8099948528 te 8466045457 email lictsrikant@gmail.com life insurance corporation of india seosrika ntht@gmail .com thathineni srikanth insurance advisor agent code no. 0316164y life insurance corporation of india br. off. lic office, trimulgherry, sec’bad - 500 016. add. borabanda, hyderabad - 500 018. lictsrikant8099948528.blogspot.in, interviewsinhyderabad.blogspot.in facebook.com/lictsrikant8099948528, facebook.com/thathineni.srikanth.9 promote your business online pybo


In [10]:
from spacy import displacy

In [11]:
displacy.serve(doc,style='ent')

d:\datascienceanywhere\udemy\document_scanner\notes\docuscan\lib\site-packages\spacy\displacy\__init__.py:97: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [12]:
displacy.render(doc,style='ent')

### Tagging

In [68]:
docjson = doc.to_json()
docjson.keys()

dict_keys(['text', 'ents', 'tokens'])

In [73]:
doc_text = docjson['text']

In [86]:
datafram_tokens = pd.DataFrame(docjson['tokens'])
datafram_tokens['token'] = datafram_tokens[['start','end']].apply(
    lambda x:doc_text[x[0]:x[1]] , axis = 1)
datafram_tokens.head(10)

,id,start,end,token
0,0,0,4,cell
1,1,5,15,8099948528
2,2,16,18,te
3,3,19,29,8466045457
4,4,30,35,email
5,5,36,57,lictsrikant@gmail.com
6,6,58,62,life
7,7,63,72,insurance
8,8,73,84,corporation
9,9,85,87,of


In [88]:
right_table = pd.DataFrame(docjson['ents'])[['start','label']]
datafram_tokens = pd.merge(datafram_tokens,right_table,how='left',on='start')

In [91]:
datafram_tokens.fillna('O',inplace=True)
datafram_tokens.head(10)

,id,start,end,token,label
0,0,0,4,cell,O
1,1,5,15,8099948528,B-PHONE
2,2,16,18,te,O
3,3,19,29,8466045457,B-PHONE
4,4,30,35,email,O
5,5,36,57,lictsrikant@gmail.com,B-EMAIL
6,6,58,62,life,B-ORG
7,7,63,72,insurance,I-ORG
8,8,73,84,corporation,I-ORG
9,9,85,87,of,I-ORG


In [122]:
# join lable to df_clean dataframe
df_clean['end'] = df_clean['text'].apply(lambda x: len(x)+1).cumsum() - 1 
df_clean['start'] = df_clean[['text','end']].apply(lambda x: x[1] - len(x[0]),axis=1)

In [124]:
# inner join with start 
dataframe_info = pd.merge(df_clean,datafram_tokens[['start','token','label']],how='inner',on='start')

In [126]:
dataframe_info.tail(10)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
41,5,1,6,3,1,6,495,474,53,21,95,018.,307,303,018,O
42,5,1,6,3,2,1,46,506,430,27,86,"lictsrikant8099948528.blogspot.in,",342,308,lictsrikant8099948528.blogspot.in,I-WEB
43,5,1,6,3,2,2,488,506,429,27,86,interviewsinhyderabad.blogspot.in,376,343,interviewsinhyderabad.blogspot.in,B-WEB
44,5,1,6,3,3,1,44,539,472,25,87,"facebook.com/lictsrikant8099948528,",412,377,facebook.com/lictsrikant8099948528,O
45,5,1,6,3,3,2,526,539,443,22,91,facebook.com/thathineni.srikanth.9,447,413,facebook.com/thathineni.srikanth.9,O
46,5,1,6,3,4,1,46,571,106,21,96,promote,455,448,promote,O
47,5,1,6,3,4,2,161,576,56,22,96,your,460,456,your,O
48,5,1,6,3,4,3,226,571,111,21,96,business,469,461,business,O
49,5,1,6,3,4,4,347,571,74,21,92,online,476,470,online,O
50,5,1,6,3,4,5,432,571,96,27,92,pybo,481,477,pybo,O


### Bounding Box

In [132]:
bb_df = dataframe_info.query("label != 'O' ")
img = image.copy()

for x,y,w,h,label in bb_df[['left','top','width','height','label']].values:
    x = int(x)
    y = int(y)
    w = int(w)
    h = int(h)
    
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.putText(img,str(label),(x,y),cv2.FONT_HERSHEY_PLAIN,1,(255,0,255),2)
    
    
cv2.imshow('Predictions',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [134]:
bb_df['label'] = bb_df['label'].apply(lambda x: x[2:])
bb_df.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
1,5,1,3,1,1,3,822,53,203,28,96,8099948528,15,5,8099948528,PHONE
3,5,1,3,2,1,2,822,71,203,65,96,8466045457,29,19,8466045457,PHONE
5,5,1,3,2,2,3,709,136,316,31,88,lictsrikant@gmail.com,57,36,lictsrikant@gmail.com,EMAIL
6,5,1,3,3,1,1,46,170,33,14,96,life,62,58,life,ORG
7,5,1,3,3,1,2,85,151,91,42,94,insurance,72,63,insurance,ORG


In [136]:
# group the label
class groupgen():
    def __init__(self):
        self.id = 0
        self.text = ''
        
    def getgroup(self,text):
        if self.text == text:
            return self.id
        else:
            self.id +=1
            self.text = text
            return self.id
        
grp_gen = groupgen()

In [137]:
bb_df['group'] = bb_df['label'].apply(grp_gen.getgroup)

In [146]:
# right and bottom of bounding box
bb_df[['left','top','width','height']] = bb_df[['left','top','width','height']].astype(int)
bb_df['right'] = bb_df['left'] + bb_df['width']
bb_df['bottom'] = bb_df['top'] + bb_df['height']

In [148]:
# tagging: groupby group
col_group = ['left','top','right','bottom','label','token','group']
group_tag_img = bb_df[col_group].groupby(by='group')

In [150]:
img_tagging = group_tag_img.agg({
    
    'left':min,
    'right':max,
    'top':min,
    'bottom':max,
    'label':np.unique,
    'token':lambda x: " ".join(x)
    
})

In [151]:
img_tagging

,left,right,top,bottom,label,token
group,,,,,,
1,822,1025,53,136,PHONE,8099948528 8466045457
2,709,1025,136,167,EMAIL,lictsrikant@gmail.com
3,46,374,151,193,ORG,life insurance corporation of india
4,668,1024,165,209,EMAIL,seosrika ntht@gmail .com
5,310,755,227,259,NAME,thathineni srikanth
6,399,669,271,296,DES,insurance advisor
7,47,882,395,427,ORG,life insurance corporation of india
8,46,917,506,533,WEB,lictsrikant8099948528.blogspot.in interviewsin...


In [ ]:
img_bb = image.copy()
for l,r,t,b,label,token in img_tagging.values:
    cv2.rectangle(img_bb,(l,t),(r,b),(0,255,0),2)
    
    cv2.putText(img_bb,label,(l,t),cv2.FONT_HERSHEY_PLAIN,1,(255,0,255),2)
    
    
cv2.imshow('Bounding Box BusinessCard',img_bb)
cv2.waitKey(0)
cv2.destroyAllWindows()